## IMPORT DATASET FROM GOOGLE DRIVE

In [5]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/gdrive')

# Read the CSV file from Google Drive
data = pd.read_csv('/content/gdrive/My Drive/ADS/201709301651_masters_portal.csv')

# Display the first few rows of the DataFrame
data.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,country_name,country_code,university_name,university_rank,program_name,program_type,deadline,duration,language,tution_1_currency,...,tution_2_money,tution_2_type,tuition_price_specification,start_date,ielts_score,structure,academic_req,facts,city,program_url
0,Armenia,ARM,American University of Armenia,NaN,Economics,MSc,2004-07-18T00:00:00Z,NaN,English,EUR,...,2108.0,EU/EEA,Tuition (Year),2018-09-01 00:00:00,6.5,['Quantitative Methods for Economists (Mathema...,"<section id=""AcademicRequirements""> <h2>Academ...",['Starting in 2018-09-01 00:00:00 You can...,['Yerevan'],http://www.mastersportal.eu/studies/71101/econ...
1,Armenia,ARM,American University of Armenia,NaN,Political Science and International Affairs,Master,2031-07-18T00:00:00Z,24 months,English,EUR,...,2500.0,National,Tuition (Year),2018-08-22 00:00:00,6.5,NaN,"<section id=""AcademicRequirements""> <h2>Academ...",['Starting in 2018-08-22 00:00:00 You can...,['Yerevan'],http://www.mastersportal.eu/studies/71085/poli...
2,Armenia,ARM,American University of Armenia,NaN,Business Administration,MBA,2004-07-18T00:00:00Z,NaN,English,EUR,...,2499.0,EU/EEA,Tuition (Year),2018-09-01 00:00:00,6.5,['Managers with practical knowledge of account...,"<section id=""AcademicRequirements""> <h2>Academ...",['Starting in 2018-09-01 00:00:00 You can...,['Yerevan'],http://www.mastersportal.eu/studies/71102/busi...
3,Armenia,ARM,American University of Armenia,NaN,Computer and Information Science,MSc,NaN,24 months,English,EUR,...,2500.0,National,Tuition (Year),NaN,6.5,['Introduction to Object-Oriented Programming'...,"<section id=""AcademicRequirements""> <h2>Academ...",['Deadline and start date Application deadline...,['Yerevan'],http://www.mastersportal.eu/studies/71104/comp...
4,Armenia,ARM,American University of Armenia,NaN,Industrial Engineering and Systems Management,MEng,2031-07-18T00:00:00Z,24 months,English,EUR,...,2500.0,National,Tuition (Year),2018-08-22 00:00:00,6.5,"['Probability Theory', 'Analysis and Design of...","<section id=""AcademicRequirements""> <h2>Academ...",['Starting in 2018-08-22 00:00:00 You can...,['Yerevan'],http://www.mastersportal.eu/studies/71103/indu...


## CLEANING THE DATA 



In [6]:
## Setting the column Currency to upper so that we can easily compare it to the user input in our filter 
import numpy as np
# data['tution_1_currency'] = data['tution_1_currency'].apply(lambda x: x.upper() if not pd.isnull(x) else np.nan)


## Setting all string columns to lowercase so that we can easily compare them to the user input in our filters 

# create a list of columns containing string datatypes to apply the function to
cols = ['program_name', 'program_type', 'language', 'tution_1_currency', 'tution_2_type']

# apply the  function to each column in the list
data[cols] = data[cols].applymap(lambda x: x.lower() if not pd.isnull(x) else np.nan)



## Column duration: Some values are in months, some in days, some not specified. We want only month values and only a number, without the string element after to be able to compare it. 

# Extract numerical part of duration column, save it in the column 'duration_num'
data['duration_num'] = data['duration'].str.extract('(\d+)').astype(float)

# Strip string elements after numbers
data['duration'] = data['duration'].str.extract('(\d+)', expand=False)

# Convert 'days' to 'months' by devidig /30.5 (we had to use an average) and round  values. Those values will be stored directly in the 'duration' column (this will be updated with the new values)
data.loc[data['duration'].str.contains('day', na=False) , 'duration_num'] = round(data.loc[data['duration'].str.contains('day', na=False) , 'duration_num'] / 30.5)
# data['duration'] = data['duration_num'].astype(int).astype(str) + ' months'

# Drop 'duration_num' column
data = data.drop('duration_num', axis=1)



## Fixing Tution Column : As 'free' is denoted in the currency column, and then the actual tution is an NAN, lets replace those instances with 0: 
data.loc[(data['tution_1_currency'] == 'free') & (data['tution_2_money'].isna()), 'tution_2_money'] = 0




## DEFINE DATA STRUCTURE

PSEUDOCODE / OUTLINE:

1) Define the data structure:

- Create a class for the master's programs with specified attributes from dataset. 
- Create a class for students, with attributes like name, interests, preferred location, budget, and other relevant information.

In [30]:
# initialize Program class:
class MasterProgram:
  # using ** to be able to input as many arguments as we want, not having to define how many befroehand
    def __init__(self, **kwargs):
        for key, value in kwargs.items(): # iterating over all attributes that will be inserted
                                          # using items() to retriebe key-value pairs as tuple 
            setattr(self, key, value) # set attribute with the setattr() function using the key and value we previously retrieved 

    def __str__(self):
        return f"{self.program_name} at {self.university_name}"

# Create a list of MasterProgram objects:
master_programs = [MasterProgram(**row) for _, row in data.iterrows()] # iterating over each row (** again to not have to predefine the number of rows) 

##CREATE HASH TABLES

In [38]:
def create_hashtable(data):
    return {f"{row['program_name']}_{row['university_name']}": MasterProgram(**row) for _, row in data.iterrows()}

## DEFINE POSSIBLE FILTERS

In [44]:
# Define filter functions
def filter_country(programs, country):
    filtered_programs = [program for program in programs if program.country_name.lower() == country.lower()]
    if not filtered_programs:
        print(f"No programs found in {country}. Here are the available options for countries:")
        options = set([program.country_name for program in programs])
        print('\n'.join(options))
    else:
        return filtered_programs

def filter_program_name(programs, program_name):
    filtered_programs = [program for program in programs if program_name.lower() in program.program_name.lower()]
    if not filtered_programs:
        print(f"No programs found with '{program_name}'. Here are the available options:")
        options = set([program.program_name for program in programs])
        print('\n'.join(options))
    else:
        return filtered_programs

def filter_program_type(programs, program_type):
    filtered_programs = [program for program in programs if program.program_type.lower() == program_type.lower()]
    if not filtered_programs:
        print(f"No programs found with '{program_type}' program type. Here are the available options:")
        options = set([program.program_type for program in programs])
        print('\n'.join(options))
    else:
        return filtered_programs

def filter_language(programs, language):
    # here, we purposefully don't check if language == language, but rather if it is in english, as many programs will have multiple languages denoted, and we also want to see the option if our input is one of the offered languages. 
    filtered_programs = [program for program in programs if str(language).lower() in str(program.language).lower()]
    if not filtered_programs:
        print(f"No programs found in {language}. Here are the available options:")
        options = set([str(program.language) for program in programs])
        print('\n'.join(options))
    else:
        return filtered_programs


def filter_ielts(programs, max_ielts_score):
  # the ielts score is a score in an english test that might be required from students. Therefore, the student will probably want to filter for if the score is x (their score) or lower, which we do here: 
    filtered_programs = [program for program in programs if program.ielts_score <= max_ielts_score]
    if not filtered_programs:
        print(f"No programs found with a minimum IELTS score of {max_ielts_score}.")
    else:
        return filtered_programs

def filter_uni_name(programs, university_name):
    filtered_programs = [program for program in programs if program.university_name.lower() == university_name.lower()]
    if not filtered_programs:
        print(f"No programs found at {university_name}. Here are the available options:")
        options = set([program.university_name for program in programs])
        print('\n'.join(options))
    else:
        return filtered_programs

def filter_uni_rank(programs, start_rank=None, end_rank=None):
    """Filter programs by university rank range."""
    # if the user does not input any start rank, we will use 0 as a starting point
    if start_rank is None:
        start_rank = 0
    # if the user does not input any last rank, we will use infinity as an ending point(maximum will be all programs in our dataset)
    if end_rank is None:
        end_rank = float('inf')
    filtered_programs = [program for program in programs if start_rank <= program.university_rank <= end_rank]
    if not filtered_programs:
        print(f"No programs found between ranks {start_rank} and {end_rank}.")
    else:
        return filtered_programs

def filter_duration(programs, duration):
    """Filter programs by duration."""
    # Checking if the user inputs a 
    if not str(duration).isdigit():
        print("Warning: Please enter a valid number for duration.")
        return
        
    filtered_programs = [program for program in programs if str(program.duration).lower() == str(duration).lower()]
    if not filtered_programs:
        print(f"No programs found with a duration of {duration} months. Here are the available options:")
        options = set([str(program.duration) for program in programs])
        print('\n'.join(options))
    else:
        return filtered_programs

def filter_currency(programs, currency):
    """Filter programs by currency."""
    valid_currencies = set(data['tution_1_currency'].unique())
    # Add or remove valid currencies as needed
    if not isinstance(programs, list):
        raise TypeError("The 'programs' argument must be a list.")
    if not isinstance(currency, str):
        raise TypeError("The 'currency' argument must be a string.")
    currency = currency.lower()  # Convert to lowercase to be able to compare it
    if currency not in valid_currencies:
        raise ValueError(f"'{currency}' is not a valid currency. Valid options are: {', '.join(valid_currencies)}")
    return [program for program in programs if isinstance(program.tution_1_currency, str) and program.tution_1_currency.lower() == currency]


def filter_tuition_cost(programs, min_cost, max_cost):
    """Filter programs by tuition cost range."""
    if not all(isinstance(cost, (int, float)) for cost in [min_cost, max_cost]):
        print("Invalid input. Minimum and maxmimum cost must be a number.")
        return None
    else:
        filtered_programs = [program for program in programs if min_cost <= program.tution_1_money <= max_cost]
        return filtered_programs

def filter_tuition_type(programs, tuition_type):
    """Filter programs by tuition type."""
    unique_tuition_types = set(str(program.tution_1_type).lower() for program in programs)
    if tuition_type.lower() not in unique_tuition_types:
        print(f"Invalid tuition type. Available options: {', '.join(unique_tuition_types)}")
        return None
    else:
        filtered_programs = [program for program in programs if str(program.tution_1_type).lower() == tuition_type.lower()]
        return filtered_programs


In [40]:
# Define a ranking function (rank after university rank)
def rank_programs(programs):
    if programs is None:
        return []
    else: 
        return sorted(programs, key=lambda x: x.university_rank)


In [41]:
# Define a function to create a box around text
import textwrap
def create_box(text):
    lines = text.split('\n')
    width = max(len(line) for line in lines)
    res = [f'┌{"─" * width}┐']
    for line in lines:
        res.append(f'│{line:{width}}│')
    res.append(f'└{"─" * width}┘')
    return '\n'.join(res)

In [42]:
# Main Function 

def main(): 

  # Display available filters
  print("Available Filters:\n" +
        "1. Country\n" +
        "2. Program Name\n" +
        "3. Program Type\n" +
        "4. Language\n" +
        "5. IELTS Score Required\n" +
        "6. University Name\n" +
        "7. University Rank\n" +
        "8. Duration\n" +
        "9. Tuition Currency\n" +
        "10. Tuition Cost\n" +
        "11. Tuition Type")


  # Get the filter numbers from the user
  filter_numbers = input("Enter the above indicated numbers of the filters you would like to use, separated by commas: ")

  filter_numbers = [int(num.strip()) for num in filter_numbers.split(',')]

  filtered_programs = master_programs

  # Apply the chosen filters
  for filter_number in filter_numbers:
      if filter_number == 1:
          country_filter = input("Enter the country: ")
          filtered_programs = filter_country(filtered_programs, country_filter)
      elif filter_number == 2:
          program_name_filter = input("Enter the program name: ")
          filtered_programs = filter_program_name(filtered_programs, program_name_filter)
      elif filter_number == 3:
          program_type_filter = input("Enter the program type: ")
          filtered_programs = filter_program_type(filtered_programs, program_type_filter)
      elif filter_number == 4:
          language_filter = input("Enter the language: ")
          filtered_programs = filter_language(filtered_programs, language_filter)
      elif filter_number == 5:
          min_ielts_score = float(input("Enter the maximum IELTS score that the program should require: "))
          filtered_programs = filter_ielts(filtered_programs, min_ielts_score)
      elif filter_number == 6:
          university_name = input("Enter the University Name:  ")
          filtered_programs = filter_uni_name(filtered_programs, university_name)
      elif filter_number == 7:
          start_rank = int(input("Enter the starting rank: "))
          end_rank = int(input("Enter the ending rank: "))
          filtered_programs = filter_uni_rank(filtered_programs, start_rank, end_rank)
      elif filter_number == 8:
          duration_filter = input("Please enter the desired duration in Months: ")
          filtered_programs = filter_duration(filtered_programs, duration_filter)
      elif filter_number == 9:
          currency_filter = input("Enter the currency: ")
          filtered_programs = filter_currency(filtered_programs, currency_filter)
      elif filter_number == 10:
          min_cost = float(input("Enter the minimum tuition cost (currencies may vary, best is to also filter currency ): "))
          max_cost = float(input("Enter the maximum tuition cost(currencies may vary, best is to also filter currency ): "))
          filtered_programs = filter_tuition_cost(filtered_programs, min_cost, max_cost)
      elif filter_number == 11:
          tuition_type_filter = input("Enter the tuition type: ")
          filtered_programs = filter_tuition_type(filtered_programs, tuition_type_filter)
      else:
          print(f"Invalid filter number ({filter_number}). Skipping this filter.")


  # Rank the filtered programs
  ranked_programs = rank_programs(filtered_programs)

  # Ask user how many programs they want to see (in case that there is at least 1 matching program)
  if len(ranked_programs)>=1: 
    n = input("Enter the maximum number of Master's Programs you would like to be shown: ")
    try: 
      float(n)
      n = int(n)
    except ValueError: 
      print("n is not a number! Please enter an integer number")


  # Display  fitting programs to the user:
  if isinstance(n, int):
    for rank, program in enumerate(ranked_programs[:n], start=1):
        program_details = f"{rank}. {program} (University Rank: {program.university_rank})"
        if len(program_details)<1: 
          print(f"\nUnfortunately, no Program matches your criteria. Change the filters and try again!\n")
        else:
          print(f"\nThe List Below shows {n} Adequate Universities According To Your Preferences, sorted after University Rank:")
          boxed_details = create_box(textwrap.fill(program_details, width=80))
          print(boxed_details)

  else: 
    pass # this is the case that the in put n is not an integer, we only pass as we have already printed the error message
 


In [43]:
if __name__ == "__main__":  
  main()

Available Filters:
1. Country
2. Program Name
3. Program Type
4. Language
5. IELTS Score Required
6. University Name
7. University Rank
8. Duration
9. Tuition Currency
10. Tuition Cost
11. Tuition Type
Enter the above indicated numbers of the filters you would like to use, separated by commas: 1
Enter the country: Germany
Enter the maximum number of Master's Programs you would like to be shown: 3

The List Below shows 3 Adequate Universities According To Your Preferences, sorted after University Rank:
┌──────────────────────────────────────────────────────────────────────┐
│1. aerospace engineering at FH Aachen - University of Applied Sciences│
│(University Rank: nan)                                                │
└──────────────────────────────────────────────────────────────────────┘

The List Below shows 3 Adequate Universities According To Your Preferences, sorted after University Rank:
┌─────────────────────────────────────────────────────────────────────────────┐
│2. biomedical